# Azure AI Agent service - Connected agents

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [5]:
import datetime
import os
import sys
import time

from azure.ai.agents import AgentsClient
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import BingGroundingTool, ConnectedAgentTool, FilePurpose, FileSearchTool, ListSortOrder, MessageRole
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

## Settings

In [6]:
load_dotenv(".env")

False

In [7]:
endpoint = os.getenv("PROJECT_ENDPOINT")
credential = DefaultAzureCredential()

project_client = AIProjectClient(endpoint=endpoint, credential=credential)

## Agents definition

### 1. Stocks price agent

In [8]:
bing_connection = project_client.connections.get(name=os.environ["BING_CONNECTION_NAME"])
BING_CONN_ID = bing_connection.id
model = os.getenv("MODEL_DEPLOYMENT_NAME")
bing = BingGroundingTool(connection_id=BING_CONN_ID)

In [9]:
agents_client = project_client.agents
stocksprice_agent = agents_client.create_agent(
    model=model,
    name="stocks-price-agent-demo",
    instructions="Retrieve the current stock price for a specified company.",
    tools=bing.definitions,
    headers={"x-ms-enable-preview": "true"},
    description="Stocks price agent (demo)",
    temperature=0.7,
)

In [10]:
print(f"Created agent, ID: {stocksprice_agent.id}")

Created agent, ID: asst_rq6DsLkoTvbibzFcf2q5l8HI


### 2. Currency agent

In [11]:
currency_convert_agent = agents_client.create_agent(
    model=model,
    name="currency-convert-agent-demo",
    instructions="Retrieve the current currency exchange rate.",
    tools=bing.definitions,
    headers={"x-ms-enable-preview": "true"},
    description="Currency convert agent (demo)",
    temperature=0.7,
)

In [12]:
print(f"Created agent, ID: {currency_convert_agent.id}")

Created agent, ID: asst_PN9TkeQkAbeDpJasNxRzYEbp


### 3. Main agent with connected agents

In [13]:
connected_agent_stocks = ConnectedAgentTool(
    id=stocksprice_agent.id,
    name="stockspriceagentconnection",
    description="Get the stock price of a company")

In [14]:
connected_agent_currency = ConnectedAgentTool(
    id=currency_convert_agent.id,
    name="currencyconvertagentconnection",
    description="Convert an amount into another currency")

In [15]:
main_agent_demo = agents_client.create_agent(
    model=model,
    name="main-agent-demo",
    instructions="You are the main AI agent",
    description="Main agent (demo)",
    tools=[
        connected_agent_stocks.definitions[0],
        connected_agent_currency.definitions[0]
    ],
)

In [16]:
print(f"Created agent, ID: {main_agent_demo.id}")

Created agent, ID: asst_gULrRxDjoYojVDgI5GpxsZPb


## List of agents

In [17]:
agents_list = [stocksprice_agent, currency_convert_agent, main_agent_demo]

for agent in agents_list:
    print(
        f"Agent Details:\n"
        f"📛 Name        : {agent.name}\n"
        f"🆔 ID          : {agent.id}\n"
        f"🧠 Model       : {agent.model}\n"
        f"📜 Instructions: {agent.instructions}\n"
    )

Agent Details:
📛 Name        : stocks-price-agent-demo
🆔 ID          : asst_rq6DsLkoTvbibzFcf2q5l8HI
🧠 Model       : gpt-4.1
📜 Instructions: Retrieve the current stock price for a specified company.

Agent Details:
📛 Name        : currency-convert-agent-demo
🆔 ID          : asst_PN9TkeQkAbeDpJasNxRzYEbp
🧠 Model       : gpt-4.1
📜 Instructions: Retrieve the current currency exchange rate.

Agent Details:
📛 Name        : main-agent-demo
🆔 ID          : asst_gULrRxDjoYojVDgI5GpxsZPb
🧠 Model       : gpt-4.1
📜 Instructions: You are the main AI agent



### Quick test

### Example 1

In [18]:
prompt = "What is microsoft's share price?"

In [20]:
# Step 1: Create a thread
thread = agents_client.threads.create()
print(f"🧵 [1] Created thread  | ID: {thread.id}")

# Step 2: Send the user's message
message = agents_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"🏃 [3] Created run     | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"\n🔍 Step ID details: {step.id}, Type: {step.type}")
    print(f"Details step: {step}")

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = agents_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"💡 → {msg.role}: {last_text.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        idx = 1
        for annotation in msg.url_citation_annotations:
            print(f"🔗 Source {idx}: [{annotation.url_citation.title}]({annotation.url_citation.url})")
            idx += 1

print("\033[0m")  # Reset color

🧵 [1] Created thread  | ID: thread_zl8eCpgsB1cSxVfGBpLtSRQ1
💬 [2] Created message | ID: msg_MiIKPC9zjO3WTz13nhc4FNTv
🏃 [3] Created run     | ID: run_fnjwpO1mOXRUFy48m0sFQlmu
⏳ [4] Polling run status...
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.COMPLETED

🔍 Step ID details: step_cH7mNHGugiGkU7rrGElhCOFW, Type: RunStepType.MESSAGE_CREATION
Details step: {'id': 'step_cH7mNHGugiGkU7rrGElhCOFW', 'object': 'thread.run.step', 'created_at': 1756184461, 'run_id': 'run_fnjwpO1mOXRUFy48m0sFQlmu', 'assistant_id': 'asst_gULrRxDjoYojVDgI5GpxsZPb', 'thread_id': 'thread_zl8eCpgsB1cSxVfGBpLtSRQ1', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1756184462, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_Z7lDA3bIy3vQQkgLZMXWYF02'}}, 'usage': {'prompt_tokens': 362, 'completion_tokens': 61, 'total_

### Example 2

In [21]:
prompt = "Convert this into € and in Japanese Yen"

In [22]:
# Step 1: Use a thread
#thread = project_client.threads.create()
print(f"🧵 [1] Using existing thread  | ID: {thread.id}")

# Step 2: Send the user's message
message = agents_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"🏃 [3] Created run     | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"\n🔍 Step ID details: {step.id}, Type: {step.type}")
    print(f"Details step: {step}")

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = agents_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"💡 → {msg.role}: {last_text.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        idx = 1
        for annotation in msg.url_citation_annotations:
            print(f"🔗 Source {idx}: [{annotation.url_citation.title}]({annotation.url_citation.url})")
            idx += 1

print("\033[0m")  # Reset color

🧵 [1] Using existing thread  | ID: thread_zl8eCpgsB1cSxVfGBpLtSRQ1
💬 [2] Created message | ID: msg_JnpWhHqUZzumVOQ2zfuwQWZ1
🏃 [3] Created run     | ID: run_DKQh5tGXJqemntNTZW5jw8aL
⏳ [4] Polling run status...
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.COMPLETED

🔍 Step ID details: step_rhm19fvWDmhxCLz4788jgMYX, Type: RunStepType.MESSAGE_CREATION
Details step: {'id': 'step_rhm19fvWDmhxCLz4788jgMYX', 'object': 'thread.run.step', 'created_at': 1756184507, 'run_id': 'run_DKQh5tGXJqemntNTZW5jw8aL', 'assistant_id': 'asst_gULrRxDjoYojVDgI5GpxsZPb', 'thread_id': 'thread_zl8eCpgsB1cSxVfGBpLtSRQ1', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1756184507, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_Qt1AQEhw788ZvLOSEIkAPB8g'}}, 'usage': {'prompt_tokens': 551, 'completion_tokens': 79, 'total_tokens': 630, 'prompt_tok

In [24]:
for agent in agents_list:
   agents_client.delete_agent(agent.id)
   print(f"Deleted agent, agent ID: {agent.id}")

Deleted agent, agent ID: asst_rq6DsLkoTvbibzFcf2q5l8HI
Deleted agent, agent ID: asst_PN9TkeQkAbeDpJasNxRzYEbp
Deleted agent, agent ID: asst_gULrRxDjoYojVDgI5GpxsZPb
